In [1]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
import umap
import matplotlib
import mygene
%matplotlib inline
import pickle
import sklearn
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from sentence_transformers import SentenceTransformer
import os
# Specify the working directory
os.chdir('/Users/david/Desktop/FinetuneEmbed')

# import sentence_transformers
plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9A4710B9-0DA3-36BB-9129-645F282E64B2> /Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <ECC148AF-20FF-3EEE-BC75-4DD3E7455393> /Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvi

In [2]:
# with open(f"./data/embeddings/GPT_3_5_gene_embeddings_fromGenePT.pickle", "rb") as fp:
#     GPT_3_5_gene_embeddings = pickle.load(fp)

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## Long- vs short- range TFs
The input data used here are downloaded from Chen et al. (2020) (link: https://www-nature-com.stanford.idm.oclc.org/articles/s41467-020-16106-x).

In [3]:
# prepare the input data
with open("./data/long_vs_shortTF/train_data.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("./data/long_vs_shortTF/test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

In [4]:
M = train_data['dataloader']

In [12]:
X_train = []
y_train = []

for texts, labels in train_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    print(texts)
    print(len(texts))
    embeddings = sbert_model.encode(texts[0][0], convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_train.extend(embeddings)
    y_train.extend(labels)

X_test = []
y_test = []
for texts, labels in test_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts[0][0], convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_test.extend(embeddings)
    y_test.extend(labels)

X_train = [tensor.to('cpu').numpy() for tensor in X_train]
y_train = [value.numpy() for value in y_train]
X_test = [tensor.to('cpu').numpy() for tensor in X_test]
y_test = [value.numpy() for value in y_test]

[("Official Symbol JUNBprovided by HGNC Official Full Name JunB proto - oncogene, AP - 1 transcription factor subunitprovided by HGNC Primary source HGNC: HGNC: 6205 See related Ensembl: ENSG00000171223 MIM: 165161; AllianceGenome: HGNC: 6205 Gene type protein coding RefSeq status VALIDATED Organism Homo sapiens Lineage Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Family hominidae; Homo Also known as AP - 1 Compendious Enables sequence - specific double - stranded DNA binding activity. Involved in irrefutable regulation of transcription by RNA polymerase II. Located in nucleoplasm. Character of transcription factor AP - 1 complex. Biomarker of Hodgkin ' s lymphoma and anaplastic large cell lymphoma. [provided by Alliance of Genome Resources, Apr 2022] Orthologs mouse all",)]
1
('Official Symbol ZEB1provided by HGNC Official Full Name zinc finger E-box binding homeobox 1provided by HGNC Primary

KeyboardInterrupt: 

In [8]:
# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_score_logistic = logistic_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_logistic)
roc_auc = auc(fpr, tpr)
# Print ROC AUC scores
print(f"Logistic Regression ROC AUC: {roc_auc:.4f}")
# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_score_rf = random_forest_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_rf)
roc_auc = auc(fpr, tpr)
print(f"Random Forest ROC AUC: {roc_auc:.4f}")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (53793,) + inhomogeneous part.

## Dosage sensitive vs insensitive TFs

In [6]:
# prepare the input data
with open("./data/DosageSensitivity/train_data.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("./data/DosageSensitivity/test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

In [7]:
X_train = []
y_train = []

for texts, labels in train_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_train.extend(embeddings)
    y_train.extend(labels)

X_test = []
y_test = []
for texts, labels in test_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_test.extend(embeddings)
    y_test.extend(labels)

X_train = [tensor.to('cpu').numpy() for tensor in X_train]
y_train = [value.numpy() for value in y_train]
X_test = [tensor.to('cpu').numpy() for tensor in X_test]
y_test = [value.numpy() for value in y_test]

In [8]:
# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_score_logistic = logistic_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_logistic)
roc_auc = auc(fpr, tpr)
# Print ROC AUC scores
print(f"Logistic Regression ROC AUC: {roc_auc:.4f}")
# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_score_rf = random_forest_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_rf)
roc_auc = auc(fpr, tpr)
print(f"Random Forest ROC AUC: {roc_auc:.4f}")

Logistic Regression ROC AUC: 0.8734
Random Forest ROC AUC: 0.8896


## Bivalent vs. lys4

In [9]:
# prepare the input data
with open("./data/MethylationState/bivalent_vs_lys4/train_data.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("./data/MethylationState/bivalent_vs_lys4/test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

X_train = []
y_train = []

for texts, labels in train_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_train.extend(embeddings)
    y_train.extend(labels)

X_test = []
y_test = []
for texts, labels in test_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_test.extend(embeddings)
    y_test.extend(labels)

X_train = [tensor.to('cpu').numpy() for tensor in X_train]
y_train = [value.numpy() for value in y_train]
X_test = [tensor.to('cpu').numpy() for tensor in X_test]
y_test = [value.numpy() for value in y_test]

# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_score_logistic = logistic_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_logistic)
roc_auc = auc(fpr, tpr)
# Print ROC AUC scores
print(f"Logistic Regression ROC AUC: {roc_auc:.4f}")
# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_score_rf = random_forest_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_rf)
roc_auc = auc(fpr, tpr)
print(f"Random Forest ROC AUC: {roc_auc:.4f}")

Logistic Regression ROC AUC: 0.9868
Random Forest ROC AUC: 0.9655


## Bivalent vs. no methyl

In [10]:
# prepare the input data
with open("./data/MethylationState/bivalent_vs_no_methyl/train_data.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("./data/MethylationState/bivalent_vs_no_methyl/test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

X_train = []
y_train = []

for texts, labels in train_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_train.extend(embeddings)
    y_train.extend(labels)

X_test = []
y_test = []
for texts, labels in test_data['dataloader']:
    # Get SBERT embeddings for the batch of texts
    embeddings = sbert_model.encode(texts, convert_to_tensor=True)
    
    # Collect embeddings and labels
    X_test.extend(embeddings)
    y_test.extend(labels)

X_train = [tensor.to('cpu').numpy() for tensor in X_train]
y_train = [value.numpy() for value in y_train]
X_test = [tensor.to('cpu').numpy() for tensor in X_test]
y_test = [value.numpy() for value in y_test]

# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_score_logistic = logistic_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_logistic)
roc_auc = auc(fpr, tpr)
# Print ROC AUC scores
print(f"Logistic Regression ROC AUC: {roc_auc:.4f}")
# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
y_score_rf = random_forest_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score_rf)
roc_auc = auc(fpr, tpr)
print(f"Random Forest ROC AUC: {roc_auc:.4f}")

Logistic Regression ROC AUC: 0.9013
Random Forest ROC AUC: 0.8717
